In [ ]:
import json
import pickle
from itertools import chain

import numpy as np
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
%matplotlib inline

This notebook contains summary plots of the synaptic cooperativity experiments for the two cell types and for three different conditions:

- passive cell
- active cell without sodium channels (simulating a TTX condition)
- active cell

#### Load the data

In [ ]:
cell_type = 'a-thorny'
data = {'passive': pickle.load(open('synaptic_cooperativity_' + cell_type + '_passive_without_TTX.pkl', 'rb')),
        'active_with_TTX': pickle.load(open('synaptic_cooperativity_' + cell_type + '_active_with_TTX.pkl', 'rb')),
        'active': pickle.load(open('synaptic_cooperativity_' + cell_type + '_active_without_TTX.pkl', 'rb'))}
n_spines = data['active_with_TTX']['config']['n_spines']
spike_times = data['active_with_TTX']['spike_times']

#### Measure the amplitude ratio when the input is synaptic

In [ ]:
N = len(data)
y_lim = np.array([0, -100])
fig,ax = plt.subplots(1, N, figsize=(4*N, 4), sharex=True, sharey=True)
for i, (expt_type, expt) in enumerate(data.items()):
    idx, = np.where((expt['t'] > 990) & (expt['t'] < 1200))
    EPSP_spine = np.max(expt['Vspine'][0,idx]) - expt['Vspine'][0,-1]
    EPSP_dend = np.max(expt['Vdend'][0,idx]) - expt['Vdend'][0,-1]
    if expt['Vspine'][0,idx[0]] < y_lim[0]:
        y_lim[0] = expt['Vspine'][0,idx[0]]
    if np.max(expt['Vspine'][0,idx]) > y_lim[1]:
        y_lim[1] = np.max(expt['Vspine'][0,idx])
    title = expt_type.replace('_', ' ').upper()
    print('{}:'.format(title))
    print('   Amplitude ratio: {:.3f}'.format(expt['AR']))
    print('     Dendritic Rin: {:.0f} MOhm'.format(expt['R']['dend']))
    print('    Spine neck Rin: {:.0f} MOhm'.format(expt['R']['neck']))
    ax[i].plot(expt['t'][idx], expt['Vspine'][0,idx], 'k', label='Spine')
    ax[i].plot(expt['t'][idx], expt['Vdend'][0,idx], 'r', label='Dendrite')
    ax[i].set_xlabel('Time (ms)')
    ax[i].set_title(title)
ax[0].set_ylim(y_lim + np.diff(y_lim) / np.array([-20,20]))
ax[0].set_ylabel('Vm (mV)')
ax[-1].legend(loc='best');

#### Plot the amplitude of the dendritic and somatic EPSPs and of the NMDA conductance as a function of the number of synaptic inputs

In [ ]:
n = 1 + np.arange(n_spines)
fig,(ax1,ax2,ax3) = plt.subplots(1, 3, figsize=(17,5), sharex=True)
col = [[0, 0, 0], [.7,0,.7], [0,.6,0]]
lw = 1.5
for i, (expt_type, expt) in enumerate(data.items()):
    if expt_type == 'active':
        continue
    lbl = expt_type.replace('_', ' ')
    ax1.plot(n, expt['dV_dend'], 'o-', color=col[i], linewidth=lw-0.5, markeredgewidth=lw,
             markerfacecolor='w', markersize=7, label=lbl)
    ax2.plot(n, expt['dV_soma'], 'o-', color=col[i], linewidth=lw-0.5, markeredgewidth=lw,
             markerfacecolor='w', markersize=7, label=lbl)
    ax3.plot(n, expt['dG_AMPA'], 's-', color=col[i], linewidth=lw-0.5, markeredgewidth=lw,
             markerfacecolor='w', markersize=5, label='AMPA')
    ax3.plot(n, expt['dG_NMDA'], 'o-', color=col[i], linewidth=lw-0.5, markeredgewidth=lw,
             markerfacecolor='w', markersize=7, label='NMDA')
ax1.set_xlabel('Input number')
ax1.set_ylabel('Dendritic deflection (mV)')
ax2.set_xlabel('Input number')
ax2.set_ylabel('Somatic deflection (mV)')
ax3.set_xlabel('Input number')
ax3.set_ylabel('Conductance (nS)')
ax1.legend(loc='best');

#### Plot the time course of the dendritic and somatic EPSPs for increasing number of synaptic inputs

In [ ]:
cmap = plt.get_cmap('rainbow', n_spines)
window = [10, 100]
fig,ax = plt.subplots(2, N, figsize=(17,10), sharex=True, sharey=True)
for i, (expt_type, expt) in enumerate(data.items()):
    for j, spk in enumerate(spike_times[0]):
        t = expt['t']
        idx = (t > spk - window[0]) & (t < spk + window[1])
        ax[0,i].set_title(expt_type.replace('_', ' ').upper())
        ax[0,i].plot(t[idx] - spk, expt['Vdend'][0,idx], color=cmap(j))
        ax[1,i].plot(t[idx] - spk, expt['Vsoma'][idx], color=cmap(j))
    ax[-1,i].set_xlabel('Time since input (ms)')
ax[0,0].set_ylabel('Dendritic voltage (mV)')
ax[1,0].set_ylabel('Somatic voltage (mV)');